In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from Webscrape import Webscrape

In [14]:
base_url = 'https://www.zorgkaartnederland.nl/'

In [15]:
scrape = Webscrape(base_url=base_url)

In [16]:
facility = 'tandartsenpraktijk'

In [17]:
results = scrape.get_filter_result(facility=facility)

In [18]:
scrape.get_filter_links(results=results)

AttributeError: 'Webscrape' object has no attribute 'get_filter_links'

## Debugging

In [3]:
test_url = 'https://www.zorgkaartnederland.nl/overzicht/organisatietypes'

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
response = requests.get(url=test_url)
response.status_code

200

In [6]:
soup = BeautifulSoup(response.content, 'html.parser')

In [9]:
search_list = soup.find('div', class_='search-list')
search_list

<div class="search-list">
<div class="row">
<div class="col-md-6">
<a class="filter-radio" href="/medisch-pedicurepraktijk">(Medisch) Pedicurepraktijk <span class="filter-radio__counter">(5031)</span></a>
<a class="filter-radio" href="/abortuskliniek">Abortuskliniek <span class="filter-radio__counter">(18)</span></a>
<a class="filter-radio" href="/apotheek">Apotheek <span class="filter-radio__counter">(1945)</span></a>
<a class="filter-radio" href="/arbo-en-reintegratiebedrijf">Arbo- en reïntegratiebedrijf <span class="filter-radio__counter">(531)</span></a>
<a class="filter-radio" href="/artsenpraktijk-voor-alternatieve-geneeskunde">Artsenpraktijk voor alternatieve geneeskunde <span class="filter-radio__counter">(392)</span></a>
<a class="filter-radio" href="/astmacentrum">Astmacentrum <span class="filter-radio__counter">(5)</span></a>
<a class="filter-radio" href="/audiologisch-centrum">Audiologisch centrum <span class="filter-radio__counter">(45)</span></a>
<a class="filter-radio" h

In [11]:
result_list = search_list.find_all('div', class_='col-md-6')
result_list

[<div class="col-md-6">
 <a class="filter-radio" href="/medisch-pedicurepraktijk">(Medisch) Pedicurepraktijk <span class="filter-radio__counter">(5031)</span></a>
 <a class="filter-radio" href="/abortuskliniek">Abortuskliniek <span class="filter-radio__counter">(18)</span></a>
 <a class="filter-radio" href="/apotheek">Apotheek <span class="filter-radio__counter">(1945)</span></a>
 <a class="filter-radio" href="/arbo-en-reintegratiebedrijf">Arbo- en reïntegratiebedrijf <span class="filter-radio__counter">(531)</span></a>
 <a class="filter-radio" href="/artsenpraktijk-voor-alternatieve-geneeskunde">Artsenpraktijk voor alternatieve geneeskunde <span class="filter-radio__counter">(392)</span></a>
 <a class="filter-radio" href="/astmacentrum">Astmacentrum <span class="filter-radio__counter">(5)</span></a>
 <a class="filter-radio" href="/audiologisch-centrum">Audiologisch centrum <span class="filter-radio__counter">(45)</span></a>
 <a class="filter-radio" href="/belangenorganisatie">Belangen

In [12]:
len(result_list)

2